<a href="https://colab.research.google.com/github/MarieLycksell/2022_Bioinformatics/blob/main/labPhylogeny_dendropyExercise_bioinfo2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DentroPy (https://dendropy.org/) is a python library for phylogeny, and we're going to use it to do a little bit of phylogenetic computations in python.

First we need to install it to our Colab instance and import it to our python environment.

In [ ]:
!python3 -m pip install -U dendropy
import dendropy as den

Upload a file with all the aligned sequences for the "dino" exercise in the quiz.

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Now we can read in the content of this fasta file using dendropy, separating the content into one object containing the taxon lables and one containing the DNA sequences.

In [ ]:
taxa = den.TaxonNamespace() #Empty list of taxa
dna = den.DnaCharacterMatrix.get(
    path="dino_all.fasta", #Specifying which file to read
    schema="fasta", #Specifying the type of file
    taxon_namespace=taxa) #Specifying that the taxon lables from the file should be added to the list we named "taxa"

print(taxa)

The character matrix data objects of dendropy are an interesting data type in that it has a behaviour similar to both a list and a dictionary. You can call an entry from the character matrix using either the index of the entry in the matrix or using the taxon lable of that entry.

Edit this block of code to check that you are indeed getting the same DNA sequence when retrieving it from the character matrix using index or taxon label.

In [ ]:
sl = str(dna[3]) #Creating a string of a character matrix entry using the index of the entry
sd = str(dna['Human']) #Creating a string of a character matrix entry using the taxonlable of the entry

if CHANGE HERE: #Formulate the logical test to check if the string are identical
  print('The same sequence was retrieved')
else:
  print('Different sequences were retrived')


As we have access to all of the sequences, we can utilize python to calculate the distance matrix for us. To do this we are going to utilize the Numpy library (https://numpy.org/), which is a common python library to use for doing math.

In [6]:
import numpy as np

Numpy handles numbers in a format called arrays. Numpy arrays can roughtly be thought of as vectors from linear algebra. Matrices can also be expressed as arrays by having an array where the entries are themselves arrays. When calling a specific entry from an array that could be described as a 2D matrix, one first specify the desired row, followed by the desired column.

In [ ]:
#Making an expample array
a = np.asarray([[1,2,3],
                [4,5,6],
                [7,8,9]])

print('Our "matrix"')
print(a)

print('The first entry in a is itself an array, the first row in our matrix')
print(a[0]) #Remember that Python starts counting with 0

print('The first element in the third row is')
print(a[2,0]) #First number specifiec desired row, the second the desired column

Now that we are familiar with how to handle a matrix expressed as a numpy array we can start constructing a distance matrix. We are going to define a function for calculating the distance matrix, then we'll let this function operate on our character matrix to get our distance matrix.

Update the code so that the Hamming distance is calculated.

In [ ]:
def hamming(characterMatrix):
  n = len(characterMatrix) #Setting n as the nr of taxa
  D = np.zeros((n,n)) #Making a matrix of zeors with correct dimensions
  taxa = [] #List of taxa
  for i in range(0,n-1):
    seq1 = str(characterMatrix[i])
    for j in range(i+1,n):
      seq2 = str(characterMatrix[j])
      dist = 0 #Variable to measure the distance between seq1 and seq2. Initial assumption that they are the same - will be updated
      if len(seq1) != len(seq2): #Checking if the sequences have the same length
        print('Error! Different length of sequences in alignment')
        break
      else:
        for x, y in zip(seq1, seq2): #Iterating over seq1 and seq2 in tandem
          if x != y: #If a position in seq1 and seq2 is different
            CHANGE HERE #updating the Hamming distance of the sequences
      D[i,j] = dist
      D[j,i] = dist
  return D

#Using our function for calculating Hamming distances on our character matrix of dna sequences to get a distance matrix called "mat"
mat = hamming(dna)
mat

Another distance measure that is fairly straight forward to calculate in Python when you only know the sequences is the fraction of mismatches. 

Write a function which calculates the distance matrix using fraction of mismatches. *Hint: How can you change the function above to use fraction of mismatches instead of Hamming distance?*

In [ ]:
def misMatchFrac(characterMatrix):
  #Write your function here

mat2 = misMatchFrac(dna)
mat2

Dendropy can't read distance matrices from numpy arrays, it has its own format for distance matrices. To get the distance matrix into a format that Dendropy understand we are going to save our numpy array together with the taxon lables as a comma separated csv file that we then read in with Dendropy. It is a bit of a workaround, but all solutions can't be beautiful.

In [ ]:
l = [] #Creating an empty list
for i in range(0,len(taxa)+1):
  l.append([])
  if i == 0:
    l[i].append('')
    for j in range(0,len(taxa)):
      l[i].append(taxa.labels()[j])
  else:
    l[i].append(taxa.labels()[i-1])
    for d in mat[i-1]:
      l[i].append(str(d))

np.savetxt('dino_hamming.csv', l, delimiter=',', fmt='%s')

!cat dino_hamming.csv # !cat prints the file we just wrote. cat stands for "concatenate" and it is a bash command rather than a python command, hence why in Colab it is proceeded by ! in order to use it. Bash commands are used on Unix systems (e.g. Linux and Mac computers) to interact with the computer through text commands.

#Loading the csv file with Dendropy to get it as a Dendropy distance matrix
dmat = den.PhylogeneticDistanceMatrix.from_csv(src=open("dino_hamming.csv"), delimiter=",")

With the distance matrix in the Dendropy format we can now have Dendropy use it to construct various phylogenetic trees.

In [ ]:
#UPGMA tree
upgma_tree = dmat.upgma_tree()

print('UPGMA tree:')
upgma_tree.print_plot()
print(upgma_tree.as_string("newick"))

#Neighbour-Joning tree
nj_tree = dmat.nj_tree()

print('NJ tree:')
nj_tree.print_plot()
print(nj_tree.as_string("newick"))


Use the printed Newick format of the **NJ tree** and compare the tree drawn by Dendropy with the tree drawn by TreeDyn (http://www.phylogeny.fr/one_task.cgi?task_type=treedyn). When pasting the Newick format tree, be sure to include all the paranthesises and the semi-colon ; at the end.

What information is not conveyed in the tree as drawn by Dendropy despite this information having been calculated?

_________________________________________________________
# Optional exercise

Previously we have applied the UPGMA algorithm by hand, which was a fair bit of work. As the number of taxa increas or the algorithm used becomes more complex, programming is very useful as you only need to do the work of writing how the algorithm should be executed. With a function programmed that can execute the alogithm, you just need to feed data to the function rather than you yourself iterating through the algorithm by hand. However, programming a function to execute the algorithm may not be trivial.

Challenge yourself to write an algorithm that constructs an UPGMA phylogenetic tree when given a distance matrix.

*It is probably a good idea to break down the problem into smaller parts rather. You can start by writing functions for the separate mathematical steps in the UPGMA algorithm, i.e. calculating branch lengths and new distances for the updated distance matrix.*

In [ ]:
#Write an UPGMA algorithm 